In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
X = np.random.rand(1817, 100, 72)
y = np.random.rand(1817)

In [3]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
train_dataset = tf.data.Dataset.from_tensor_slices((X_train.astype(np.float32), 
                                                    y_train.astype(np.float32)))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val.astype(np.float32), 
                                                  y_val.astype(np.float32)))

In [4]:
from afqinsight.nn.tf_models import cnn_lenet

/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model = cnn_lenet(input_shape=X_train.shape[1:], n_classes=1)
model.compile(loss='mean_squared_error')

In [6]:
from afqinsight.augmentation import jitter, time_warp, scaling, magnitude_warp


In [7]:
def tf_aug(X_in, scaler=1/25.):
    X_out = np.zeros_like(X_in)
    for channel in range(X_in.shape[-1]):
        this_X = X_in[..., channel][np.newaxis, ..., np.newaxis]
        scale = np.abs(np.max(this_X) - np.min(this_X)) * scaler
        this_X = jitter(this_X, sigma=scale)
        this_X = scaling(this_X, sigma=scale)
        this_X = time_warp(this_X, sigma=scale)
        X_out[..., channel] = this_X[0, ..., 0]
    return X_out

In [8]:
def augment_this(X_in, y_in):
    X_out = tf.numpy_function(tf_aug, [X_in], tf.float32)    
    return X_out, y_in

In [ ]:
train_dataset = train_dataset.map(augment_this)
train_dataset = train_dataset.batch(32)
val_dataset = val_dataset.batch(32)

model.fit(train_dataset, epochs=100, validation_data=val_dataset)

Epoch 1/100
46/46 [==============================] - 35s 717ms/step - loss: 0.3266 - val_loss: 0.3651
Epoch 2/100
12/46 [======>.......................] - ETA: 25s - loss: 0.2969

In [ ]:
debug